# Spooky Books Author Prediction

Langkah langkah yang dilakukan:
1. Memuat data
2. Melakukan eksplorasi singkat
3. Preprocessing teks menggunakan TF-IDF
4. Melatih model Logistic Regression multinomial
5. Memprediksi probabilitas dan membuat file submission


## 1. Import Libraries
Pertama, kita import library yang diperlukan:
- `pandas` untuk manipulasi data
- `TfidfVectorizer` dari `sklearn` untuk konversi teks ke vektor TF-IDF
- `LogisticRegression` dari `sklearn` untuk model klasifikasi

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

## 2. Load Data
Memuat file CSV berikut yang sudah di-upload:
- `train.csv`: data latih berisi 3 kolom (`id`, `text`, `author`)
- `test.csv`: data uji berisi 2 kolom (`id`, `text`)
- `sample_submission.csv`: contoh format file submission

In [2]:
# Load data
train = pd.read_csv('./train/train.csv')
test = pd.read_csv('./test/test.csv')
sample_sub = pd.read_csv('./sample_submission/sample_submission.csv')

# Tampilkan ukuran data
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Sample submission shape: {sample_sub.shape}")

Train shape: (19579, 3)
Test shape: (8392, 2)
Sample submission shape: (8392, 4)


**Penjelasan**:
- `train.shape` menunjukkan jumlah baris dan kolom di data latih.
- Data latih memiliki label `author` yang akan kita prediksi.
- Data uji belum memiliki label, dan kita akan mengisi probabilitas setiap kelas.

## 3. Prepare Features and Labels
Pisahkan kolom teks sebagai fitur dan kolom penulis sebagai target.

In [3]:
# Fitur (X) dan target (y)
X_train = train['text']
y_train = train['author']
X_test = test['text']

## 4. Text Preprocessing with TF-IDF
Kita gunakan TF-IDF untuk mengubah teks menjadi vektor numerik.
- `max_features=5000`: Batasi dimensi vektor
- `ngram_range=(1,2)`: Gunakan unigram dan bigram
- `stop_words='english'`: Hapus kata umum bahasa Inggris

In [4]:
# Inisialisasi TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),
    stop_words='english'
)

# Fit pada data latih dan transformasi pada data latih + uji
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF selesai. Bentuk matriks fitur:")
print("X_train_tfidf:", X_train_tfidf.shape)
print("X_test_tfidf:", X_test_tfidf.shape)

TF-IDF selesai. Bentuk matriks fitur:
X_train_tfidf: (19579, 5000)
X_test_tfidf: (8392, 5000)


## 5. Train Logistic Regression Model
Melatih model `LogisticRegression` dengan solver `saga`:
- `multi_class='multinomial'` untuk klasifikasi multi-kelas
- `max_iter=1000` untuk memastikan konvergensi
- `random_state=42` agar hasil bisa direproduksi

In [5]:
# Inisialisasi dan pelatihan model
model = LogisticRegression(
    multi_class='multinomial',
    solver='saga',
    max_iter=1000,
    random_state=42
)
model.fit(X_train_tfidf, y_train)

print("Model berhasil dilatih.")

C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model berhasil dilatih.


## 6. Predict Probabilities and Create Submission
Menghitung probabilitas prediksi untuk setiap kelas (`EAP`, `HPL`, `MWS`) pada data uji dan membuat DataFrame submission.

In [6]:
# Prediksi probabilitas
probs = model.predict_proba(X_test_tfidf)
classes = model.classes_

# Buat DataFrame submission
submission = pd.DataFrame(probs, columns=classes)
submission.insert(0, 'id', test['id'])
submission = submission[['id', 'EAP', 'HPL', 'MWS']]

# Tampilkan 5 baris pertama
submission.head()

,id,EAP,HPL,MWS
0,id02310,0.278435,0.085087,0.636478
1,id24541,0.785862,0.128351,0.085787
2,id00134,0.466790,0.465800,0.067410
3,id27757,0.739007,0.212420,0.048573
4,id04081,0.775821,0.150436,0.073742


## 7. Save Submission File
Menyimpan file `submission.csv` sesuai format kompetisi:
```csv
id,EAP,HPL,MWS
idXXXXX,0.33,0.33,0.33
```

In [7]:
# Simpan ke CSV
submission.to_csv('./sample_submission/submission.csv', index=False)
print("File submission.csv berhasil disimpan.")

File submission.csv berhasil disimpan.
